In [1]:
import pickle
from pathlib import Path
import numpy as np
import pandas as pd
root = Path(".")

In [2]:
article_1 = pd.read_csv('articles1.csv')
news_content = article_1['content'][:500]

In [4]:
documents = list()

for file in news_content:
    ans = list()
    for i in range(len(file)):
        if file[i].isalnum():
            ans.append(file[i].lower())
        else:
            ans.append(' ')
    
    res = list()
    for i in range(len(ans) - 1):
        if ans[i] == ans[i + 1] and ans[i] == ' ':
            continue
        else:
            res.append(ans[i])
    
    if len(ans) > 0:
        res.append(ans[-1])
        documents.append(''.join(res))

In [5]:
def get_shingle_set(documents, k):
    shingles = set()
    shingle_doc_id = dict()

    for j in range(len(documents)):
        for i in range(len(documents[j]) - k + 1):
            cur_shingle = documents[j][i : i + k]

            shingles.add(cur_shingle)
            
            if shingle_doc_id.get(cur_shingle) == None:
                shingle_doc_id[cur_shingle] = [j]
            else:
                shingle_doc_id[cur_shingle].append(j)
    
    return (shingles, shingle_doc_id)

In [6]:
for k in range(2, 11):
    print(k, len(get_shingle_set(documents, k)[0]))

2 994
3 9223
4 47147
5 149523
6 358639
7 669031
8 1040926
9 1432704
10 1802365


In [7]:
K = 3
shingles, shingle_doc_id = get_shingle_set(documents, K)
shingles = sorted(list(shingles))

In [8]:
incident_matrix = np.zeros(shape=(len(shingles), len(documents)))
incident_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
shingle_id = dict()
for i in range(len(shingles)):
    shingle_id[shingles[i]] = i

for shingle, doc_ids in shingle_doc_id.items():
    for doc_id in doc_ids:
        incident_matrix[shingle_id[shingle]][doc_id] = 1

incident_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [10]:
my_path = root / "Pickled_files" / "Incident_Matrix"
dbfile = open(my_path, 'wb')
pickle.dump(incident_matrix, dbfile) 
dbfile.close()

my_path = root / "Pickled_files" / "Shingles"
dbfile = open(my_path, 'wb')
pickle.dump(shingles, dbfile) 
dbfile.close()

my_path = root / "Pickled_files" / "Shingle_id"
dbfile = open(my_path, 'wb')
pickle.dump(shingle_id, dbfile) 
dbfile.close()

my_path = root / "Pickled_files" / "documents"
dbfile = open(my_path, 'wb')
pickle.dump(documents, dbfile) 
dbfile.close()

In [11]:
for i in range(len(documents)): 
    matrix = incident_matrix[0 : len(incident_matrix), i]
    print(np.count_nonzero(matrix == 1), matrix.shape[0])

1347 9223
2669 9223
2231 9223
2449 9223
1133 9223
497 9223
1132 9223
2285 9223
1931 9223
1374 9223
1589 9223
1975 9223
1866 9223
1120 9223
1878 9223
1812 9223
1779 9223
1767 9223
1178 9223
788 9223
1393 9223
1230 9223
1710 9223
1877 9223
1735 9223
1452 9223
1846 9223
1883 9223
1397 9223
2302 9223
1418 9223
1646 9223
1401 9223
958 9223
1069 9223
1317 9223
1803 9223
1649 9223
1232 9223
1686 9223
1747 9223
1709 9223
1740 9223
1762 9223
1075 9223
1697 9223
1309 9223
858 9223
1609 9223
1601 9223
1981 9223
1701 9223
906 9223
1368 9223
1832 9223
1945 9223
1200 9223
1466 9223
1450 9223
1397 9223
1248 9223
1665 9223
1816 9223
1779 9223
1506 9223
2854 9223
1122 9223
1332 9223
1278 9223
1332 9223
1292 9223
3110 9223
1531 9223
1519 9223
1349 9223
1429 9223
856 9223
923 9223
1672 9223
808 9223
1528 9223
832 9223
1212 9223
1509 9223
1800 9223
1154 9223
1441 9223
1655 9223
1437 9223
3055 9223
1682 9223
1522 9223
1502 9223
1244 9223
1748 9223
1593 9223
1256 9223
884 9223
1535 9223
1025 9223
1523 9223
